<a href="https://colab.research.google.com/github/jeonghojo00/HousingPricePrediction/blob/main/HousingPrice_Prophet_byZip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly

## Step 0. Connect to Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Step 1. Define Functions

### Load Data

In [60]:
def load_data():
    colab_notebook = "/content/drive/My Drive/Colab Notebooks/"
    zillow_dataset = "Zillow/AllHomesZipPrices.csv"
    df = pd.read_csv(colab_notebook+zillow_dataset)
    return df

### Get City Dataframe

In [61]:
def get_city_df(df, state, city):
    state_df = df[df['state'] == state]
    city_df = state_df[state_df['city'] == city]

    return city_df

### Get House Price Prediction

In [62]:
def get_HousingPricePrediction(city_df, zip, period = 24):
    zip_df = city_df[city_df['zip']==zip].loc[:,('ds', 'y')]
    md = Prophet(interval_width=0.95)
    model = md.fit(zip_df)
    future = model.make_future_dataframe(periods=period, freq='MS') #freq changes frequency of dates. Default Daily
    pred_y = model.predict(future)

    return md, pred_y

## Step 2. Run Codes

### Load Dataset

In [63]:
df = load_data()

### Get City Dataframe

In [64]:
state = 'TX'
city = 'Houston'
city_df = get_city_df(df, state, city)
zipCodes = city_df['zip'].unique()

print(zipCodes)

[77084 77036 77077 77095 77082 77007 77057 77083 77070 77042 77063 77081
 77008 77072 77089 77015 77040 77339 77024 77088 77099 77090 77055 77044
 77004 77064 77080 77060 77006 77009 77065 77079 77035 77096 77054 77056
 77092 77074 77025 77073 77093 77034 77019 77075 77598 77018 77027 77041
 77014 77049 77021 77087 77047 77091 77017 77066 77005 77043 77022 77071
 77023 77016 77067 77045 77098 77058 77062 77076 77345 77033 77061 77053
 77020 77038 77069 77086 77026 77039 77048 77051 77002 77030 77011 77012
 77059 77028 77013 77031 77029 77003 77336 77085 77037 77078 77068 77094
 77032 77050 77046 77010]


### Select one of zip codes

In [65]:
zip = 77084

### Get House Price Prediction

In [66]:
# Train and predict using Prophet model
md, pred_y = get_HousingPricePrediction(city_df, zip, period = 24)

# Plot
plot_plotly(md, pred_y)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
